In [ ]:
from aes_ground_truth import WI_HEX_I4_TO_I43
from engine_context import CKKS_EngineContext
from aes_main_process import engine_initiation
from aes_transform_zeta import zeta_to_int
from aes_key_scheduling import key_initiation_fixed, _extract_word_hex, _extract_bytes_hex
import numpy as np

delta = [1 * 2048, 2 * 2048, 3 * 2048, 4 * 2048, 5 * 2048, 6 * 2048, 7 * 2048, 8 * 2048, 9 * 2048, 10 * 2048, 11 * 2048, 12 * 2048, 13 * 2048, 14 * 2048, 15 * 2048]

engine_context = engine_initiation(signature=1, mode='parallel', use_bootstrap=True, thread_count = 16, device_id = 0, fixed_rotation=True, delta_list=delta) 

print("slot_count: ", engine_context.get_slot_count())

engine = engine_context.get_engine()
public_key = engine_context.get_public_key()

key_zeta_hi, key_zeta_lo = key_initiation_fixed()

key_zeta_hi = engine.encrypt(key_zeta_hi, public_key, level=10)
key_zeta_lo = engine.encrypt(key_zeta_lo, public_key, level=10)

hex_byte = _extract_bytes_hex(engine_context, key_zeta_hi, key_zeta_lo)
print("key_zeta_hi", hex_byte)

enc_key_hi_list = []
enc_key_lo_list = []

mask_row_0 = np.concatenate((np.ones(4 * 2048), np.zeros(12 * 2048)))
mask_row_1 = np.concatenate((np.zeros(4 * 2048), np.ones(4 * 2048), np.zeros(8 * 2048)))
mask_row_2 = np.concatenate((np.zeros(8 * 2048), np.ones(4 * 2048), np.zeros(4 * 2048)))
mask_row_3 = np.concatenate((np.zeros(12 * 2048), np.ones(4 * 2048))) 

row_hi_0 = engine.multiply(key_zeta_hi, mask_row_0)
row_hi_1 = engine.multiply(key_zeta_hi, mask_row_1)
row_hi_2 = engine.multiply(key_zeta_hi, mask_row_2)
row_hi_3 = engine.multiply(key_zeta_hi, mask_row_3)

row_lo_0 = engine.multiply(key_zeta_lo, mask_row_0)
row_lo_1 = engine.multiply(key_zeta_lo, mask_row_1)
row_lo_2 = engine.multiply(key_zeta_lo, mask_row_2)
row_lo_3 = engine.multiply(key_zeta_lo, mask_row_3)


print("row_hi_0", _extract_bytes_hex(engine_context, row_hi_0, row_lo_0))
print("row_hi_1", _extract_bytes_hex(engine_context, row_hi_1, row_lo_1))
print("row_hi_2", _extract_bytes_hex(engine_context, row_hi_2, row_lo_2))
print("row_hi_3", _extract_bytes_hex(engine_context, row_hi_3, row_lo_3))

# compare add and multiply
row_hi_add = engine.add(row_hi_0, row_hi_1)
row_hi_add = engine.add(row_hi_add, row_hi_2)
row_hi_add = engine.add(row_hi_add, row_hi_3)

row_lo_add = engine.add(row_lo_0, row_lo_1)
row_lo_add = engine.add(row_lo_add, row_lo_2)
row_lo_add = engine.add(row_lo_add, row_lo_3)

hex_byte_add = _extract_bytes_hex(engine_context, row_hi_add, row_lo_add)
print("row_add", hex_byte_add)

row_hi_multiply = engine.multiply(row_hi_0, row_hi_1, engine_context.get_relinearization_key())
row_hi_multiply = engine.multiply(row_hi_multiply, row_hi_2, engine_context.get_relinearization_key())
row_hi_multiply = engine.multiply(row_hi_multiply, row_hi_3, engine_context.get_relinearization_key())

row_lo_multiply = engine.multiply(row_lo_0, row_lo_1, engine_context.get_relinearization_key())
row_lo_multiply = engine.multiply(row_lo_multiply, row_lo_2, engine_context.get_relinearization_key())
row_lo_multiply = engine.multiply(row_lo_multiply, row_lo_3, engine_context.get_relinearization_key())

hex_byte_multiply = _extract_bytes_hex(engine_context, row_hi_multiply, row_lo_multiply)
print("row_multiply", hex_byte_multiply)

print("row_add == row_multiply", hex_byte_add == hex_byte_multiply)
print("row_add == key_zeta", hex_byte_add == hex_byte)
print("row_multiply == key_zeta", hex_byte_multiply == hex_byte)

In [ ]:
from engine_context import CKKS_EngineContext
from aes_main_process import engine_initiation
from aes_transform_zeta import zeta_to_int
from aes_key_scheduling import key_initiation_fixed, _extract_word_hex, _extract_bytes_hex, _rot_word, _sub_word, _rcon_xor, _xor
import numpy as np

mask1 = np.concatenate([np.ones(8 * 2048, dtype=np.uint8), np.zeros(8 * 2048, dtype=np.uint8)])
mask2 = np.concatenate([np.ones(4 * 2048, dtype=np.uint8), np.zeros(12 * 2048, dtype=np.uint8)])


num_array = np.array([0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F])
num_array = np.tile(num_array, 2048)

engine_context = engine_initiation(signature=1, mode='parallel', use_bootstrap=True, thread_count = 16, device_id = 0)

enc_num_array = engine_context.encrypt(num_array, engine_context.get_public_key())

enc_masked_num_array_1 = engine_context.multiply(enc_num_array, mask1)
enc_masked_num_array_2 = engine_context.multiply(enc_num_array, mask2)

dec_masked_num_array_1 = engine_context.decrypt(enc_masked_num_array_1, engine_context.get_secret_key())
dec_masked_num_array_2 = engine_context.decrypt(enc_masked_num_array_2, engine_context.get_secret_key())

for i in range(16):
    idx = i * 2048
    print("dec_masked_num_array_1[idx:idx+8]", dec_masked_num_array_1[idx:idx+4])
    print("dec_masked_num_array_2[idx:idx+4]", dec_masked_num_array_2[idx:idx+4])
    print("--------------------------------")

In [ ]:
from engine_context import CKKS_EngineContext
from aes_main_process import engine_initiation
from aes_transform_zeta import zeta_to_int, int_to_zeta
from aes_key_scheduling import key_initiation_fixed, _extract_word_hex, _extract_bytes_hex, _rot_word, _sub_word, _rcon_xor, _xor
import numpy as np

mask1 = np.concatenate([np.ones(8 * 2048, dtype=np.uint8), np.zeros(8 * 2048, dtype=np.uint8)])
mask2 = np.concatenate([np.ones(4 * 2048, dtype=np.uint8), np.zeros(12 * 2048, dtype=np.uint8)])


num_array = np.array([0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F])
num_array = np.repeat(num_array, 2048)

engine_context = engine_initiation(signature=1, mode='parallel', use_bootstrap=True, thread_count = 16, device_id = 0)

num_array = int_to_zeta(num_array)
enc_num_array = engine_context.encrypt(num_array, engine_context.get_public_key())

enc_masked_num_array_1 = engine_context.multiply(enc_num_array, mask1)
enc_masked_num_array_2 = engine_context.multiply(enc_num_array, mask2)

dec_masked_num_array_1 = engine_context.decrypt(enc_masked_num_array_1, engine_context.get_secret_key())
dec_masked_num_array_2 = engine_context.decrypt(enc_masked_num_array_2, engine_context.get_secret_key())

dec_masked_num_array_1 = zeta_to_int(dec_masked_num_array_1)
dec_masked_num_array_2 = zeta_to_int(dec_masked_num_array_2)

for i in range(16):
    idx = i * 2048
    print("dec_masked_num_array_1[idx:idx+8]", dec_masked_num_array_1[idx:idx+4])
    print("dec_masked_num_array_2[idx:idx+4]", dec_masked_num_array_2[idx:idx+4])
    print("--------------------------------")

In [ ]:
from engine_context import CKKS_EngineContext
from aes_main_process import engine_initiation
from aes_transform_zeta import zeta_to_int, int_to_zeta
from aes_key_scheduling import key_initiation_fixed, _extract_word_hex, _extract_bytes_hex, _rot_word, _sub_word, _rcon_xor, _xor
import numpy as np

mask1 = np.concatenate([np.zeros(4 * 2048, dtype=np.uint8), np.ones(12 * 2048, dtype=np.uint8)])
mask2 = np.concatenate([np.ones(4 * 2048, dtype=np.uint8), np.zeros(12 * 2048, dtype=np.uint8)])

num_array = np.array([0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F])
num_array = np.repeat(num_array, 2048)

engine_context = engine_initiation(signature=1, mode='parallel', use_bootstrap=True, thread_count = 16, device_id = 0)
engine = engine_context.get_engine()

# split to hi and lo
num_array_hi = ((num_array >> 4) & 0x0F).astype(np.uint8)
num_array_lo = (num_array & 0x0F).astype(np.uint8)

num_array_hi = int_to_zeta(num_array_hi)
num_array_lo = int_to_zeta(num_array_lo)

enc_num_array_hi = engine.encrypt(num_array_hi, engine_context.get_public_key())
enc_num_array_lo = engine.encrypt(num_array_lo, engine_context.get_public_key())

enc_masked_num_array_hi_1 = engine.multiply(enc_num_array_hi, mask1)
enc_masked_num_array_lo_1 = engine.multiply(enc_num_array_lo, mask1)

enc_masked_num_array_hi_2 = engine.multiply(enc_num_array_hi, mask2)
enc_masked_num_array_lo_2 = engine.multiply(enc_num_array_lo, mask2)

rotated_num_array_hi_2, rotated_num_array_lo_2 = _rot_word(engine_context,enc_masked_num_array_hi_2, enc_masked_num_array_lo_2)
subbed_num_array_hi_2, subbed_num_array_lo_2 = _sub_word(engine_context, rotated_num_array_hi_2, rotated_num_array_lo_2)

enc_num_array_hi_2 = engine.add(subbed_num_array_hi_2, enc_masked_num_array_hi_1)
enc_num_array_lo_2 = engine.add(subbed_num_array_lo_2, enc_masked_num_array_lo_1)

dec_num_array_hi_2 = engine.decrypt(enc_num_array_hi_2, engine_context.get_secret_key())
dec_num_array_lo_2 = engine.decrypt(enc_num_array_lo_2, engine_context.get_secret_key())

dec_num_array_hi_2 = zeta_to_int(dec_num_array_hi_2)
dec_num_array_lo_2 = zeta_to_int(dec_num_array_lo_2)

dec_num_array_hi_2 = (dec_num_array_hi_2 << 4) & 0xFF
dec_num_array_lo_2 = dec_num_array_lo_2 & 0xFF

dec_num_array = dec_num_array_hi_2 | dec_num_array_lo_2

for i in range(16):
    idx = i * 2048
    print("dec_num_array[idx:idx+4]", [hex(x) for x in dec_num_array[idx:idx+4]])
    print("--------------------------------")